## University of Kaiserslautern


# Welcome to Enterprise Data Science_Covid-19 Prototyping_Final Project.
(First Dashboard Notebook)

* This project is carried out by following the lecture videos on the online platform of the University of Kaiserslautern.
* Goal of the project is to realize best practices of data science by applying a cross industry standard process(CRISP_Model) on real data project.
* Data source is provided from Johns Hopkins University data repository on Githup.
 
#### submitted to:PD DR.-ING.Frank Kienle                                               
#### Lecture Enterprise Data Science SS 2020   

#### submitted by: Mohamed Adel Abdelrahman
#### Mtrkl No:411343
#### Date: 03.09.2020

# CRISP Model Frame work
![CRISP_DM](..\reports\figures\CRISP_DM.png)

# 1. Collect Initial Data (Johns Hopkins GITHUP csv data)

In [1]:
import subprocess
import os
import pandas as pd


def JH_data_update():
    '''
    updating the entire John Hopkins dataset and save it in raw folder
    '''
    # 'git clone https://github.com/CSSEGISandData/COVID-19' only in the first time acquiring the data
    git_pull = subprocess.Popen(' git pull',   
                           cwd = os.path.dirname('../data/raw/COVID-19/'),
                           shell = True,
                           stdout = subprocess.PIPE,
                           stderr = subprocess.PIPE)
    (out, error) = git_pull.communicate()
    #return(out, error)
    data_path=r'..\data\raw\COVID-19\csse_covid_19_data\csse_covid_19_time_series\time_series_covid19_confirmed_global.csv'
    pd_raw=pd.read_csv(data_path)
    
    print("Error : " + str(error))
    print("out : " + str(out)) # printing errors during data update or clone process
    
    return pd_raw.head()     # first look at the initial dataset in a DataFrame form the
    
JH_data_update()

Error : b'From https://github.com/CSSEGISandData/COVID-19\n   7dd00337..7ad80c0c  web-data   -> origin/web-data\n'
out : b'Already up to date.\n'


,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,8/24/20,8/25/20,8/26/20,8/27/20,8/28/20,8/29/20,8/30/20,8/31/20,9/1/20,9/2/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,38054,38070,38113,38129,38140,38143,38162,38165,38196,38243
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,8605,8759,8927,9083,9195,9279,9380,9513,9606,9728
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,41858,42228,42619,43016,43403,43781,44146,44494,44833,45158
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,1060,1060,1098,1098,1124,1124,1124,1176,1184,1199
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,2222,2283,2332,2415,2471,2551,2624,2654,2729,2777


# 2. Relational data model - defining a primary key

In a relational model of databases, aprimary key is a specific choice of a minimal set of attributes (columns) that uniquely specify a tuple (row) in a relation (table)

A primary key's main features are:

* it must contain a unique value for each row of data.
* it cannot contain null values.

In [7]:
import pandas as pd
import numpy as np
from datetime import datetime

def JH_data_preprocess():
    '''
    Arrange data features and remove master data like (longitude and latitude)
    to end with a relational data set with only the transactional data.
    
    '''
    
    data_path=r'..\data\raw\COVID-19\csse_covid_19_data\csse_covid_19_time_series\time_series_covid19_confirmed_global.csv'
    pd_raw=pd.read_csv(data_path)

    
    pd_data_base = pd_raw.rename(columns={'Country/Region':'country', #rename columns
                                      'Province/State': 'state'})
    pd_data_base['state']=pd_data_base['state'].fillna('no') 
                                                
    pd_data_base = pd_data_base.drop(['Lat','Long'],axis=1) # fill NaN cells (axis=1 means columns level)

    '''
    ** the coming pipeline
    1-  construct a primary key from the (date, state, country) by pushing the state and country to an index by set_index command.
    2-  transpose 'T' the matrix to get the 'date' in a column to represent the time dimention in our dataset.
    3-  merge the multiindex 2D column(country,state) in row base like pivot tables in exel with ''stack'' command.
    4- reset the index and rename the new columns
    '''
    pd_relational_model = pd_data_base.set_index (['state','country']) \
                                              .T                   \
                                              .stack(level=[0,1])  \
                                              .reset_index()       \
                                              .rename(columns={'level_0':'date',
                                                                0:'confirmed'})

    # convert the date from (str) to (date object) data type by ''astype'' command
    pd_relational_model['date']=pd_relational_model.date.astype('datetime64[ns]')
    #store our model
    pd_relational_model.to_csv('../data/processed/COVID_relational_confirmed.csv',sep=';')
    return pd_relational_model.tail()


JH_data_preprocess()

,date,state,country,confirmed
59845,2020-09-02,no,West Bank and Gaza,23875.0
59846,2020-09-02,no,Western Sahara,10.0
59847,2020-09-02,no,Yemen,1976.0
59848,2020-09-02,no,Zambia,12415.0
59849,2020-09-02,no,Zimbabwe,6638.0


# 3. Modeling (Doubling Rate - Piecewise Linear Regression and Savitzky–Golay filter)

In [10]:

import subprocess
import os
import pandas as pd
import numpy as np
from datetime import datetime
from scipy import signal
from sklearn import linear_model
reg = linear_model.LinearRegression(fit_intercept=True)



def get_doubling_time_via_regression(in_array):
    
    ''' 
    ** Clculating slop & linear regression to approximate the doubling rate
    1- as the input is data frame so we have to slice the correct column and reshape it (2D matrix)
    2- cut out a window for x days and calculate it (-1,2) piecewise for only 3 data points(window size is called 'hyber parameter'),
       as doubling rate continuasly changes over time,
    '''                        
    y = np.array(in_array)                
    X = np.arange(-1,2).reshape(-1, 1)  
                                         
    assert len(in_array)==3 # window size
    reg.fit(X,y)
    intercept=reg.intercept_
    slope=reg.coef_

    return intercept/slope  # derivative calc

def savgol_filter (df_input,column='confirmed',window=5): # before calculating regression we have to apply the filter first
   
    ''' 
    Before calculating regression we have to apply the filter first.
    Savitzky–Golay filter is applied to clean the data so we do not distort the trend.
    
    '''
    window=5, 
    degree=1  # polynomial order 
    df_result=df_input
    
    filter_in=df_input[column].fillna(0) # attention to the neutral elements here
    
    result=signal.savgol_filter(np.array(filter_in),
                           5, # window size used for filtering & try diff days(3,4)
                           1)
    df_result[column+'_filtered']=result # adding a new feature to the dataset
    return df_result

def rolling_reg(df_input,col='confirmed'):
    ''' wrapper around reg.fit function'''
   

    days_back=3
    result=df_input[col].rolling( #rolling command is moving a window across a time series
                window=days_back,
                min_periods=days_back).apply(get_doubling_time_via_regression,raw=False)
    return result

def calc_filtered_data(df_input,filter_on='confirmed'):
    '''  Calculate savgol filter and return merged data frame by applying groupby apply.

        Parameters:
        --------
        df_input: pd.DataFrame
        filter_on: str
         defines the used column
    '''

    must_contain=set(['state','country',filter_on])
    assert must_contain.issubset(set(df_input.columns)), ' Erro in calc_filtered_data not all columns in data frame' 

    df_output=df_input.copy() # we need a copy here otherwise the filter_on column will be overwritten

    pd_filtered_result=df_output[['state','country',filter_on]].groupby(['state','country']).apply(savgol_filter)

    # left merge is more safer than outer merge, as we can control our result
    
    df_output=pd.merge(df_output,pd_filtered_result[[str(filter_on+'_filtered')]],left_index=True,right_index=True,how='left')
    return df_output.copy()

def calc_doubling_rate(df_input,filter_on='confirmed'):
    ''' Calculate approximated doubling rate and return merged data frame

    '''

    must_contain=set(['state','country',filter_on]) # making sure that all columns are there
    assert must_contain.issubset(set(df_input.columns)), ' Erro in calc_filtered_data not all columns in data frame' 


    pd_DR_result= df_input.groupby(['state','country']).apply(rolling_reg,filter_on).reset_index()

    pd_DR_result=pd_DR_result.rename(columns={filter_on:filter_on+'_DR',
                             'level_2':'index'})

    #we do the merge on the index of our big table and on the index column after groupby
    df_output=pd.merge(df_input,pd_DR_result[['index',str(filter_on+'_DR')]],left_index=True,right_on=['index'],how='left')
    df_output=df_output.drop(columns=['index'])


    return df_output

def model_test():
    ''' applying the above functions respectively on the relational data model'''
    
    test_data_reg=np.array([2,4,6])
    result=get_doubling_time_via_regression(test_data_reg)
    print('the test slope is: '+str(result))
    
    
    #reading the dataset 
    pd_JH_data=pd.read_csv('../data/processed/COVID_relational_confirmed.csv',sep=';',parse_dates=[0])
    pd_JH_data = pd_JH_data.drop(['Unnamed: 0'],axis=1)
    pd_JH_data=pd_JH_data.sort_values('date',ascending=True).copy() #sorting the dates 

    # applying the above functions
    pd_result_larg=calc_filtered_data(pd_JH_data)
    print("Filtre calculations are done ")
    print("Doubling rate calculations take a little time ..... ")
    pd_result_larg=calc_doubling_rate(pd_result_larg)
    print("Doubling rate calculations are done ")
    print("Doubling rate calculations on filtered data Almost ready.......")
    pd_result_larg=calc_doubling_rate(pd_result_larg,'confirmed_filtered')
    print("Doubling rate calculations on filtered data are done")

    mask=pd_result_larg['confirmed']>100
    # 'where command' used to filter according to the mask
    pd_result_larg['confirmed_filtered_DR']=pd_result_larg['confirmed_filtered_DR'].where(mask, other=np.NaN)
    #saving the final dataset
    pd_result_larg.to_csv('../data/processed/COVID_final_set.csv',sep=';',index=False)
    print("Data saved in data/processed/COVID_final_set.csv")
    return pd_result_larg

model_test().head()

the test slope is: [2.]
Filtre calculations are done 
Doubling rate calculations take a little time ..... 
Doubling rate calculations are done 
Doubling rate calculations on filtered data Almost ready.......
Doubling rate calculations on filtered data are done
Data saved in data/processed/COVID_final_set.csv


,date,state,country,confirmed,confirmed_filtered,confirmed_DR,confirmed_filtered_DR
0,2020-01-22,Alberta,Canada,0.0,0.0,NaN,NaN
38025,2020-01-22,no,"Korea, South",1.0,0.8,NaN,NaN
38250,2020-01-22,no,Kosovo,0.0,0.0,NaN,NaN
38475,2020-01-22,no,Kuwait,0.0,0.0,NaN,NaN
38700,2020-01-22,no,Kyrgyzstan,0.0,0.0,NaN,NaN


# 4. Evaluation & visualization
* Using the dash library to plot our data model with a user interactive dashboard

In [15]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output,State
import plotly.graph_objects as go
import pandas as pd
import numpy as np
fig = go.Figure()
# Create a DataFrame from the .csv file
df_input_large=pd.read_csv(r'..\data\processed/COVID_final_set.csv',sep=';') 

app = dash.Dash() # Launch the application

# Create a Dash layout that contains a Graph component
app.layout = html.Div([
    html.H3('Feature Transformation Dashboard'),

    dcc.Markdown('''
    #  Applied Data Science on COVID-19 data

    This dashboard shows the confirmed infected COVID-19 cases for more than 100 countries,
    the doubling rate where the number of infected cases is doubled, and finally the Savitzky-Golay filter(savgol_filter),
    that is applied to both data sets to smooth the data of irregularities,
    while taking into account the domain know-how over the number of data points (days) to which the filter is applied.

    '''),

    dcc.Markdown('''
    ## Multi-Select Country for visualization
    '''),

    # country list dropdown structure (1 st dropdown )
    dcc.Dropdown(
        id='country_drop_down',
        options=[ {'label': each,'value':each} for each in df_input_large['country'].unique()],
        value=['US', 'Germany','Italy','Egypt'], # which are pre-selected
        multi=True
    ),

    dcc.Markdown('''
        ## Select Timeline of confirmed COVID-19 cases or the approximated doubling time
        '''),

    # confirmed numbers & doubling rate trend indicator dropdown menu (2 nd dropdown)
    dcc.Dropdown(
    id='doubling_time',
    options=[
        {'label': 'Timeline Confirmed ', 'value': 'confirmed'},
        {'label': 'Timeline Confirmed Filtered', 'value': 'confirmed_filtered'},
        {'label': 'Timeline Doubling Rate', 'value': 'confirmed_DR'},
        {'label': 'Timeline Doubling Rate Filtered', 'value': 'confirmed_filtered_DR'},
    ],
    value='confirmed',
    multi=False
    ),

    dcc.Graph(figure=fig, id='main_window_slope')
])

# app inputs & outputs are described in the arguments of the callback function
@app.callback(
    Output('main_window_slope', 'figure'),
    [Input('country_drop_down', 'value'),
    Input('doubling_time', 'value')])
# update the figure traces with the new selected values from the dropdown menu
def update_figure(country_list,show_doubling):

    # updating axis tags
    if 'confirmed_DR' in show_doubling:
        my_yaxis={'type':"log",
               'title':'Approximated doubling rate over 3 days'
              }
    elif 'confirmed_filtered_DR' in show_doubling:
        my_yaxis={'type':"log",
               'title':'Approximated filtered doubling rate over 3 days'
              }
    else:
        my_yaxis={'type':"log",
                  'title':'Confirmed infected people (source johns hopkins csse, log-scale)'
              }

    #updating traces
    traces = []
    for each in country_list:

        df_plot=df_input_large[df_input_large['country']==each]

        if show_doubling=='doubling_rate_filtered':
            df_plot=df_plot[['state','country','confirmed','confirmed_filtered','confirmed_DR','confirmed_filtered_DR','date']].groupby(['country','date']).agg(np.mean).reset_index()
        else:
            df_plot=df_plot[['state','country','confirmed','confirmed_filtered','confirmed_DR','confirmed_filtered_DR','date']].groupby(['country','date']).agg(np.sum).reset_index()


        traces.append(dict(x=df_plot.date,
                                y=df_plot[show_doubling],
                                mode='markers+lines',
                                opacity=0.9,
                                name=each
                        )
                )

    return {
            'data': traces,
            'layout': dict (
                width=1280,
                height=720,

                xaxis={'title':'Timeline',
                        'tickangle':-45,
                        'nticks':20,
                        'tickfont':dict(size=14,color="#7f7f7f"),
                      },

                yaxis=my_yaxis
        )
    }


#to launch multiple dash apps, i must change the port number ti differ from the second dah( here it is 8000)
app.run_server(debug=True, port=8000, use_reloader=False)


Running on http://127.0.0.1:8000/
Running on http://127.0.0.1:8000/
Running on http://127.0.0.1:8000/
Running on http://127.0.0.1:8000/
Running on http://127.0.0.1:8000/
Running on http://127.0.0.1:8000/
Running on http://127.0.0.1:8000/
Debugger PIN: 425-566-698
Debugger PIN: 425-566-698
Debugger PIN: 425-566-698
Debugger PIN: 425-566-698
Debugger PIN: 425-566-698
Debugger PIN: 425-566-698
Debugger PIN: 425-566-698
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
